In [1]:
import vrplib

In [8]:
# source: https://vrp.atd-lab.inf.puc-rio.br/index.php/en/


instance_a = vrplib.read_instance("data/A-n32-k5.vrp")

instance_b = vrplib.read_instance("data/B-n78-k10.vrp")

instance_x = vrplib.read_instance("data/X-n101-k25.vrp")